# Text Classification(NLP) using SVM and Naive Bayes 

This is a suoervised machine learning model to classfication 

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

## Step 1: Add the Corpus 

In [5]:
corpus = pd.read_csv('corpus.csv', encoding='latin-1')

corpus

,text,label
0,Stuning even for the non-gamer: This sound tr...,__label__2
1,The best soundtrack ever to anything.: I'm re...,__label__2
2,Amazing!: This soundtrack is my favorite musi...,__label__2
3,Excellent Soundtrack: I truly like this sound...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2
...,...,...
9995,A revelation of life in small town America in...,__label__2
9996,Great biography of a very interesting journal...,__label__2
9997,Interesting Subject; Poor Presentation: You'd...,__label__1
9998,Don't buy: The box looked used and it is obvi...,__label__1


## Step 2: Data-Preprocessing 

what we should do:

1. Tokenization

2. Word Stemming/Lemmatization


In [6]:
corpus.isna()

,text,label
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
9995,False,False
9996,False,False
9997,False,False
9998,False,False


In [8]:
sum(corpus['text'].isna())

0

In [9]:
corpus.columns

Index(['text', 'label'], dtype='object')

In [10]:
sum(corpus['label'].isna())

0

In [11]:
# change all the text to lower case 

corpus['text']= [word.lower() for word in corpus['text']]


In [12]:
# Tokenization

corpus['text'] = [word_tokenize(word) for word in corpus['text']]

In [13]:
corpus

,text,label
0,"[stuning, even, for, the, non-gamer, :, this, ...",__label__2
1,"[the, best, soundtrack, ever, to, anything, .,...",__label__2
2,"[amazing, !, :, this, soundtrack, is, my, favo...",__label__2
3,"[excellent, soundtrack, :, i, truly, like, thi...",__label__2
4,"[remember, ,, pull, your, jaw, off, the, floor...",__label__2
...,...,...
9995,"[a, revelation, of, life, in, small, town, ame...",__label__2
9996,"[great, biography, of, a, very, interesting, j...",__label__2
9997,"[interesting, subject, ;, poor, presentation, ...",__label__1
9998,"[do, n't, buy, :, the, box, looked, used, and,...",__label__1


In [15]:
# Remove Stop words, Non-Numeric and perfom Word 
'''
from nltk.corpus import stopwords
 set(stopwords.words('english'))
 
 stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(example_sent) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 

'''

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
stop_words
#corpus['text']= [w for w in corpus['text'] ]

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [16]:
corpus

,text,label
0,"[stuning, even, for, the, non-gamer, :, this, ...",__label__2
1,"[the, best, soundtrack, ever, to, anything, .,...",__label__2
2,"[amazing, !, :, this, soundtrack, is, my, favo...",__label__2
3,"[excellent, soundtrack, :, i, truly, like, thi...",__label__2
4,"[remember, ,, pull, your, jaw, off, the, floor...",__label__2
...,...,...
9995,"[a, revelation, of, life, in, small, town, ame...",__label__2
9996,"[great, biography, of, a, very, interesting, j...",__label__2
9997,"[interesting, subject, ;, poor, presentation, ...",__label__1
9998,"[do, n't, buy, :, the, box, looked, used, and,...",__label__1


In [17]:
text_list =[]

for item in corpus['text']:
    text_item=[]
    for word in item:
        if word not in stop_words:
            text_item.append(word)
            
    text_list.append(text_item)
    
text_list

[['stuning',
  'even',
  'non-gamer',
  ':',
  'sound',
  'track',
  'beautiful',
  '!',
  'paints',
  'senery',
  'mind',
  'well',
  'would',
  'recomend',
  'even',
  'people',
  'hate',
  'video',
  'game',
  'music',
  '!',
  'played',
  'game',
  'chrono',
  'cross',
  'games',
  'ever',
  'played',
  'best',
  'music',
  '!',
  'backs',
  'away',
  'crude',
  'keyboarding',
  'takes',
  'fresher',
  'step',
  'grate',
  'guitars',
  'soulful',
  'orchestras',
  '.',
  'would',
  'impress',
  'anyone',
  'cares',
  'listen',
  '!',
  '^_^'],
 ['best',
  'soundtrack',
  'ever',
  'anything',
  '.',
  ':',
  "'m",
  'reading',
  'lot',
  'reviews',
  'saying',
  'best',
  "'game",
  'soundtrack',
  "'",
  'figured',
  "'d",
  'write',
  'review',
  'disagree',
  'bit',
  '.',
  'opinino',
  'yasunori',
  'mitsuda',
  "'s",
  'ultimate',
  'masterpiece',
  '.',
  'music',
  'timeless',
  "'m",
  'listening',
  'years',
  'beauty',
  'simply',
  'refuses',
  'fade.the',
  'price',
  

In [19]:
corpus['text']= text_list

corpus

,text,label
0,"[stuning, even, non-gamer, :, sound, track, be...",__label__2
1,"[best, soundtrack, ever, anything, ., :, 'm, r...",__label__2
2,"[amazing, !, :, soundtrack, favorite, music, t...",__label__2
3,"[excellent, soundtrack, :, truly, like, soundt...",__label__2
4,"[remember, ,, pull, jaw, floor, hearing, :, 'v...",__label__2
...,...,...
9995,"[revelation, life, small, town, america, early...",__label__2
9996,"[great, biography, interesting, journalist, :,...",__label__2
9997,"[interesting, subject, ;, poor, presentation, ...",__label__1
9998,"[n't, buy, :, box, looked, used, obviously, ne...",__label__1


In [20]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [22]:
for index,entry in enumerate(corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    corpus.loc[index,'text_final'] = str(Final_words)

In [23]:
corpus

,text,label,text_final
0,"[stuning, even, non-gamer, :, sound, track, be...",__label__2,"['stun', 'even', 'sound', 'track', 'beautiful'..."
1,"[best, soundtrack, ever, anything, ., :, 'm, r...",__label__2,"['best', 'soundtrack', 'ever', 'anything', 're..."
2,"[amazing, !, :, soundtrack, favorite, music, t...",__label__2,"['amaze', 'soundtrack', 'favorite', 'music', '..."
3,"[excellent, soundtrack, :, truly, like, soundt...",__label__2,"['excellent', 'soundtrack', 'truly', 'like', '..."
4,"[remember, ,, pull, jaw, floor, hearing, :, 'v...",__label__2,"['remember', 'pull', 'jaw', 'floor', 'hearing'..."
...,...,...,...
9995,"[revelation, life, small, town, america, early...",__label__2,"['revelation', 'life', 'small', 'town', 'ameri..."
9996,"[great, biography, interesting, journalist, :,...",__label__2,"['great', 'biography', 'interesting', 'journal..."
9997,"[interesting, subject, ;, poor, presentation, ...",__label__1,"['interest', 'subject', 'poor', 'presentation'..."
9998,"[n't, buy, :, box, looked, used, obviously, ne...",__label__1,"['buy', 'box', 'look', 'use', 'obviously', 'ne..."


In [25]:
corpus['text_final']

0       ['stun', 'even', 'sound', 'track', 'beautiful'...
1       ['best', 'soundtrack', 'ever', 'anything', 're...
2       ['amaze', 'soundtrack', 'favorite', 'music', '...
3       ['excellent', 'soundtrack', 'truly', 'like', '...
4       ['remember', 'pull', 'jaw', 'floor', 'hearing'...
                              ...                        
9995    ['revelation', 'life', 'small', 'town', 'ameri...
9996    ['great', 'biography', 'interesting', 'journal...
9997    ['interest', 'subject', 'poor', 'presentation'...
9998    ['buy', 'box', 'look', 'use', 'obviously', 'ne...
9999    ['beautiful', 'pen', 'fast', 'delivery', 'pen'...
Name: text_final, Length: 10000, dtype: object

In [26]:
corpus['text_final'].tolist()[1]

"['best', 'soundtrack', 'ever', 'anything', 'read', 'lot', 'review', 'say', 'best', 'soundtrack', 'figure', 'write', 'review', 'disagree', 'bit', 'opinino', 'yasunori', 'mitsuda', 'ultimate', 'masterpiece', 'music', 'timeless', 'listen', 'year', 'beauty', 'simply', 'refuse', 'price', 'tag', 'pretty', 'stagger', 'must', 'say', 'go', 'buy', 'cd', 'much', 'money', 'one', 'feel', 'would', 'worth', 'every', 'penny']"

##  Step 3: Prepare Train and Test Data sets

In [27]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(corpus['text_final'],corpus['label'],test_size=0.3)

## Step 4: Encoding

In [28]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)


## Step 5: Word Vectorization

In [29]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [30]:
print(Tfidf_vect.vocabulary_)

{'stun': 4266, 'even': 1525, 'sound': 4118, 'track': 4552, 'beautiful': 382, 'paint': 3154, 'mind': 2838, 'well': 4862, 'would': 4950, 'recomend': 3594, 'people': 3224, 'hate': 2058, 'video': 4761, 'game': 1868, 'music': 2923, 'played': 3303, 'cross': 1023, 'ever': 1528, 'play': 3300, 'best': 421, 'back': 325, 'away': 316, 'crude': 1026, 'take': 4365, 'fresh': 1825, 'step': 4207, 'guitar': 2011, 'soulful': 4115, 'orchestra': 3095, 'impress': 2246, 'anyone': 202, 'care': 632, 'listen': 2614, 'soundtrack': 4119, 'anything': 203, 'read': 3559, 'lot': 2658, 'review': 3718, 'say': 3835, 'figure': 1697, 'write': 4957, 'disagree': 1225, 'bit': 441, 'ultimate': 4632, 'masterpiece': 2748, 'timeless': 4498, 'year': 4977, 'beauty': 384, 'simply': 4011, 'refuse': 3620, 'price': 3405, 'tag': 4364, 'pretty': 3400, 'must': 2929, 'go': 1936, 'buy': 589, 'cd': 664, 'much': 2914, 'money': 2872, 'one': 3076, 'feel': 1674, 'worth': 4946, 'every': 1530, 'penny': 3223, 'amaze': 148, 'favorite': 1662, 'time'

In [31]:
print(Train_X_Tfidf)

  (0, 4650)	0.12347574939565804
  (0, 4602)	0.14077091499450742
  (0, 4497)	0.07404587599271142
  (0, 4420)	0.13621150402222898
  (0, 4239)	0.16772239151312357
  (0, 4098)	0.11242584573316573
  (0, 4014)	0.11787938548558097
  (0, 3835)	0.08985803746861284
  (0, 3551)	0.13295973276894596
  (0, 3542)	0.19590161692722702
  (0, 3015)	0.1929165874349716
  (0, 2923)	0.5450162712054228
  (0, 2872)	0.10046465778999712
  (0, 2664)	0.17140905989658037
  (0, 2660)	0.1752333119814616
  (0, 2614)	0.12489471102265885
  (0, 2530)	0.15411685507306583
  (0, 2489)	0.09365539716393548
  (0, 2454)	0.21660881216919772
  (0, 2180)	0.11508207180958219
  (0, 2074)	0.1262984198399339
  (0, 2015)	0.13558048514624502
  (0, 1856)	0.14003114512492304
  (0, 1641)	0.1148409505555761
  (0, 1557)	0.1538777686218284
  :	:
  (6999, 2720)	0.08255459197786606
  (6999, 2692)	0.15514560507179065
  (6999, 2458)	0.09061168172659073
  (6999, 2180)	0.09587876056249801
  (6999, 2148)	0.14557500947520002
  (6999, 1936)	0.06833879

## Step 6: Use the ML Algorithms to Predict the outcome

### Naive Bayes

In [32]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  82.63333333333334


### SVM 

In [33]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  84.0
